In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt

***
## feature groups
- D_* = Delinquency variables
- S_* = Spend variables
- P_* = Payment variables
- B_* = Balance variables
- R_* = Risk variables

In [3]:
data = pd.read_parquet("../data/ext/amex-data-integer-dtypes-parquet-format/train.parquet")
input_cols = data.columns[2:]

categorical_cols = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
numerical_cols = [col for col in input_cols if col not in categorical_cols]

labels = pd.read_csv("../data/raw/train_labels.csv")

display(data)

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938469,0,0.008724,1.006838,0.009228,0.124035,0.0,0.004709,...,-1,-1,-1,0,0,0.0,NaN,0,0.000610,0
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936665,0,0.004923,1.000653,0.006151,0.126750,0.0,0.002714,...,-1,-1,-1,0,0,0.0,NaN,0,0.005492,0
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-05-28,0.954180,3,0.021655,1.009672,0.006815,0.123977,0.0,0.009423,...,-1,-1,-1,0,0,0.0,NaN,0,0.006986,0
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-06-13,0.960384,0,0.013683,1.002700,0.001373,0.117169,0.0,0.005531,...,-1,-1,-1,0,0,0.0,NaN,0,0.006527,0
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-07-16,0.947248,0,0.015193,1.000727,0.007605,0.117325,0.0,0.009312,...,-1,-1,-1,0,0,0.0,NaN,0,0.008126,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5531446,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,2017-11-05,0.979333,14,0.020818,0.828199,0.003487,0.090743,0.0,0.025139,...,-1,-1,-1,0,0,0.0,NaN,0,0.001498,0
5531447,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,2017-12-23,0.984907,10,0.007209,0.812610,0.005904,0.079886,0.0,0.023691,...,-1,-1,-1,0,0,0.0,NaN,0,0.008225,0
5531448,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,2018-01-06,0.983019,15,0.013151,0.815422,0.003457,0.100503,0.0,0.012343,...,-1,-1,-1,0,0,0.0,NaN,0,0.006773,0
5531449,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,2018-02-06,0.969861,15,0.009855,1.003541,0.005117,0.101802,0.0,0.008578,...,-1,-1,-1,0,0,0.0,NaN,0,0.001168,0


In [14]:
data[input_cols].mean(axis=0).sort_values(ascending=False)

S_8      1021.351741
S_13      258.162014
D_59       21.709431
B_19       14.507921
S_11       14.258346
            ...     
D_135      -0.963922
D_137      -0.964476
D_111      -0.984484
D_108      -0.994421
D_87       -0.998603
Length: 188, dtype: float64

In [15]:
data[input_cols].std(axis=0).sort_values(ascending=False)

S_8     959.537364
S_13    306.738952
B_19     28.956450
D_65     17.977417
B_4      17.346935
           ...    
R_28      0.027225
B_36      0.021064
R_23      0.017195
S_19      0.003061
B_27      0.003004
Length: 188, dtype: float64

***
## correlation analysis

In [18]:
data_feats = data.groupby("customer_ID")[numerical_cols].last().reset_index(drop=True)
data_feats

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
customer_ID,,,,,,,,,,,,,,,,,,,,,
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.934745,0,0.009382,1.007647,0.006104,0.135021,0.0,0.007174,NaN,NaN,...,-1,-1,-1,0,0,0.000000,NaN,0,0.002970,0
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5,0.880519,6,0.034684,1.004028,0.006911,0.165509,0.0,0.005068,NaN,0.060646,...,-1,-1,-1,0,0,0.000000,NaN,0,0.003169,0
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1,0.880875,0,0.004284,0.812649,0.006450,NaN,0.0,0.007196,NaN,NaN,...,-1,-1,-1,0,0,0.000000,NaN,0,0.000834,0
000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc,0.621776,0,0.012564,1.006183,0.007829,0.287766,0.0,0.009937,NaN,0.046104,...,-1,-1,-1,0,0,0.000000,NaN,0,0.005560,0
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad51ca8b8c4a24cefed,0.871900,0,0.007679,0.815746,0.001247,0.176403,0.0,0.005528,NaN,0.044671,...,-1,-1,-1,0,0,0.000000,NaN,0,0.006944,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffff41c8a52833b56430603969b9ca48d208e7c192c6a4081a6acc28cf4f8af7,0.844229,15,0.028515,1.009866,0.001928,0.128707,0.0,0.005893,NaN,0.113053,...,-1,-1,-1,0,0,0.000000,NaN,0,0.003009,0
ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fdd3e5b57cfcbee30286,0.831279,1,0.292360,0.055656,0.006953,0.194697,0.0,0.233078,NaN,0.134540,...,-1,-1,-1,0,0,0.000000,NaN,0,0.009230,0
ffff9984b999fccb2b6127635ed0736dda94e544e67e026eee4d20f680639ff6,0.800522,9,0.020563,1.007023,0.000957,0.066648,0.0,0.006314,NaN,0.049778,...,-1,-1,-1,0,0,0.000000,NaN,0,0.000340,0
